# **OpenAI Interaction**

![OpenAI](../assets/openai-logos-2025/OpenAI-logos(new)/SVGs/OpenAI-black-monoblossom.svg)

## Python Requests Library

In [1]:
import os
import requests
import json
from lib.ws_minify import ws_minify
from lib.get_random_ollama_model import get_random_ollama_model
import random

# Compose a user prompt. `textwrap.dedent` removes the common leading indentation,
# which keeps the code readable without sending leading spaces to the model.
user_prompt = ws_minify("""
    State your model and the company or lab that created you.
    Then discuss a little bit about why you were created and what are your strengths and weaknesses.
    Finally discuss situations in which it would be preferable to choose you over other models.
""")

headers = {
    "Authorization": f"Bearer {os.environ["OPENAI_API_KEY"]}",
    "Content-Type": "application/json",
}

# Endpoint for listing models
url = "https://api.openai.com/v1/models"
response = requests.get(url, headers=headers, timeout=30)

openai_models = [openai_model["id"] for openai_model in response.json().get("data", [])]

def is_chat_model(model_id: str) -> bool:
    """Heuristic filter: include chat-capable LLMs; exclude embeddings/moderation/audio."""
    s = model_id.lower()
    if any(bad in s for bad in ("embedding", "embed", "moderation", "whisper", "tts", "audio", "transcribe", "pro", "turbo", "search", "preview")):
        return False
    # Common chat families (Responses-friendly): GPT-x and o-series
    return s.startswith(("gpt-", "gpt", "o", "chatgpt"))

chat_models = sorted([m for m in openai_models if is_chat_model(m)])

current_model = random.choice(chat_models)

# Endpoint for generating text
url = "https://api.openai.com/v1/responses"

payload = {
    "model": current_model,
    "input": user_prompt
}

response = requests.post(url, headers=headers, json=payload, timeout=60)
response_json = response.json()

def extract_text(response: dict) -> str:
    # Prefer the convenience field if present; otherwise collect text from parts
    if "output_text" in response and response["output_text"]:
        return response["output_text"]
    parts = []
    for item in response.get("output", []):
        for c in item.get("content", []):
            if isinstance(c, dict) and "text" in c:
                parts.append(c["text"])
    return "".join(parts)

# Print which model we used for clarity.
print(f"{current_model}\n")

print(extract_text(response_json))

# Simple visual separator for readability.
print("\n------------------------------------------")
print("\n\n")


gpt-5-mini

I am ChatGPT, powered by OpenAI’s GPT-4o family of models (developed by OpenAI).

Why I was created
- To assist people and teams with a broad range of language- and knowledge‑based tasks: answering questions, drafting and editing text, helping with code, summarizing and explaining complex topics, tutoring, brainstorming, and handling multimodal inputs (text + images).  
- To provide an easy-to-use general-purpose AI assistant that balances usefulness, coherence, and safety for many everyday and professional workflows.

Strengths
- Broad competence across many domains (writing, reasoning, coding, explanation, translation, summarization).  
- Strong instruction-following and ability to produce well-structured outputs.  
- Multimodal capability (can work with text and images) when image input is available.  
- Fast responses and flexible conversational behavior (can be concise or detailed).  
- Designed with safety mitigations and guardrails to reduce harmful or unsafe outputs

## OpenAI Python SDK Library

In [2]:
import os
from openai import OpenAI
import json
from lib.ws_minify import ws_minify
from lib.get_random_ollama_model import get_random_ollama_model
import random

# Initialize the SDK client
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Compose a user prompt. `textwrap.dedent` removes the common leading indentation,
# which keeps the code readable without sending leading spaces to the model.
user_prompt = ws_minify("""
    State your model and the company or lab that created you.
    Then discuss a little bit about why you were created and what are your strengths and weaknesses.
    Finally discuss situations in which it would be preferable to choose you over other models.
""")

all_models = client.models.list()
openai_models = [model.id for model in all_models.data]

def is_chat_model(model_id: str) -> bool:
    """Heuristic filter: include chat-capable LLMs; exclude embeddings/moderation/audio."""
    s = model_id.lower()
    if any(bad in s for bad in (
        "embedding", "embed", "moderation", "whisper",
        "tts", "audio", "transcribe", "pro", "turbo",
        "search", "preview"
    )):
        return False
    return s.startswith(("gpt-", "gpt", "o", "chatgpt"))

chat_models = sorted([m for m in openai_models if is_chat_model(m)])
current_model = random.choice(chat_models)

def extract_text(response) -> str:
    if getattr(response, "output_text", None):
        return response.output_text
    # fallback: collect pieces
    parts = []
    for item in response.output or []:
        for c in getattr(item, "content", []):
            if hasattr(c, "text"):
                parts.append(c.text)
    return "".join(parts)

response = client.responses.create(
    model=current_model,
    input=user_prompt,
)


# Print which model we used for clarity.
print(f"{current_model}\n")

# The generated text
print(extract_text(response))

# Simple visual separator for readability.
print("\n------------------------------------------")
print("\n\n")


gpt-5-2025-08-07

- Model and creator
  - I’m ChatGPT, a GPT‑4‑class assistant created by OpenAI.

- Why I was created
  - To serve as a general-purpose reasoning and writing assistant that helps people understand information, solve problems, create content, and interact with software using natural language (and images where relevant), while aiming to be safe, useful, and broadly accessible.

- Strengths
  - Strong general reasoning and explanation: can break down complex topics into clear, structured steps.
  - Writing and editing: drafting, revising, summarizing, translating, and adapting tone for different audiences.
  - Multimodal understanding: can discuss and analyze images alongside text.
  - Coding help: explain concepts, outline approaches, and generate or refactor code in many languages.
  - Instruction following: stays on task, adapts style and level of detail, and can provide checklists or stepwise guidance.
  - Safety and privacy orientation: doesn’t access external data o